# MediaSearch Successful Searches, Positions, and Filter Usage

Four additional metrics in [T258229](https://phabricator.wikimedia.org/T258229) that we want to aggregate:

* What's the click-through rate (to quickview and from quickview to file pages)?
* What's the average position of clicked result in successful searches?
* What percentage of users click to copy the filename or wikitext to their clipboard?
* What percentage of users use a filter during their search session?

In this case I'm not going to gather data from legacy search for three reasons: 1) we already calculated a baseline for that, and 2) we'll be focused on improving MediaSearch moving forward, and 3) legacy search is a *lot* of data to sift through for little benefit.

I'm also going to measure this on a per-session basis. While we might be able to measure this on a per-search basis, we're mainly interested in understanding whether users are able to find what they're looking for during a session. It might be that the run multiple searches to refine their results, for example.

In [1]:
import datetime as dt

import pandas as pd
import numpy as np

from wmfdata import spark, mariadb

## Configuring Timestamps

We'll call the day we're gathering data for `data_day`. We're also expecting this notebook to be run the day after, which we'll call `next_day`. In order to ignore search sessions that started on the previous day, we also define that day. Lastly, we set a limit of one hour after midnight UTC as the cutoff for data. In other words, we expect search sessions to be completed within one hour.

In [2]:
next_day = dt.datetime.now(dt.timezone.utc).date()

data_day = next_day - dt.timedelta(days = 1)
previous_day = data_day - dt.timedelta(days = 1)

limit_timestamp = dt.datetime.combine(next_day, dt.time(hour = 1))

## Aggregation Tables

We define a set of tables in the Data Lake for aggregation of results.

In [4]:
search_success_table = 'nettrom_sd.mediasearch_success_aggregates'

In [5]:
create_mediasearch_table_query = '''
CREATE TABLE {table_name} (
    log_date DATE COMMENT "the date of the aggregated search counts",
    num_mediasearch_sessions BIGINT COMMENT "the number of MediaSearch sessions",
    num_used_filter BIGINT COMMENT "number of sessions that changed at least one filter",
    num_result_clicked BIGINT COMMENT "number of sessions where a result was clicked",
    num_filepage_clicked BIGINT COMMENT "number of sessions with a click to a file page",
    num_filename_copied BIGINT COMMENT "number of sessions with a click to copy the filename", 
    num_wikitext_copied BIGINT COMMENT "number of sessions with a click to copy the wikitext",
    num_media_played BIGINT COMMENT "number of sessions where media was played",
    median_position_clicked DOUBLE COMMENT "median position of a clicked result across all sessions"
)
'''

## Helper Functions

In [6]:
def make_partition_statement(start_ts, end_ts, prefix = ''):
    '''
    This takes the two timestamps and creates a statement that selects
    partitions based on `year`, `month`, and `day` in order to make our
    data gathering not use excessive amounts of data. It assumes that
    `start_ts` and `end_ts` are not more than a month apart, which should
    be a reasonable expectation for this notebook.
    
    An optional prefix can be set to enable selecting partitions for
    multiple tables with different aliases.
    
    :param start_ts: start timestamp
    :type start_ts: datetime.datetime
    
    :param end_ts: end timestamp
    :type end_ts: datetime.datetime
    
    :param prefix: prefix to use in front of partition clauses, "." is added automatically
    :type prefix: str
    '''
    
    if prefix:
        prefix = f'{prefix}.' # adds "." after the prefix
    
    # there are three cases:
    # 1: month and year are the same, output a "BETWEEN" statement with the days
    # 2: months differ, but the years are the same.
    # 3: years differ too.
    # Case #2 and #3 can be combined, because it doesn't really matter
    # if the years are the same in the month-selection or not.
    
    if start_ts.year == end_ts.year and start_ts.month == end_ts.month:
        return(f'''{prefix}year = {start_ts.year}
AND {prefix}month = {start_ts.month}
AND {prefix}day BETWEEN {start_ts.day} AND {end_ts.day}''')
    else:
        return(f'''
(
    ({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})
 OR ({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})
)''')

## The Query

A few notes:

1. Filters can be set at any point during the search session. This leads to two potential paths: 1) the filter change took place before a search was made, and thus applies to the first search following it; and 2) the filter change happened after a search, at which point a new search automatically happens to show new results based on the filter. We cannot tell from the `search_new` event that filters applied to it, but in the query below we allow `filter_change` to occur at any point during the search session. Search sessions are based on `search_new`, and we therefore assume that filters apply to searches made. The only case we're ignoring is a user setting and then resetting filters before running a search.
2. We've tested that `MAX()` returns `NULL` if all inputs are `NULL`, and `1` if one of the inputs is `1`. This isn't obvious from the Spark function documentation.
3. There's extensive use of `NULL` values to make `COUNT()` easy, rather than use `SUM()` and ones and zeroes. The previous note about how `MAX()` behaves comes into play here.
4. We've tested whether timestamps can be used to distinguish between events, and as of late March 2021 there were no difference between requiring timestamps to be different and allowing them to be equal. Once we have more data flowing in some time in April, we might want to revisit that assumption.
5. We've verified that the number of sessions counted in this query is the same as the number of sessions counted in the other notebook that also aggregates for legacy search. The `mediasearch_sessions` subquery is exactly the same so they should match, but it's good to know that nothing else in this query messes with it.

In [7]:
mediasearch_success_query = '''
WITH mediasearch_sessions AS ( -- all MediaSearch sessions started during the day of interest
    SELECT
        web_pageview_id AS session_id,
        MIN(coalesce(dt, meta.dt)) AS session_start_dt
    FROM event.mediawiki_mediasearch_interaction AS ms
    WHERE {ms_partition_statement}
    AND action = "search_new"
    GROUP BY web_pageview_id
    HAVING TO_DATE(session_start_dt) = "{today}"
),
mediasearch_filters AS ( -- used a filter in their search session
    SELECT
        DISTINCT web_pageview_id AS session_id,
        1 AS used_filter
    FROM mediasearch_sessions AS mess
    JOIN event.mediawiki_mediasearch_interaction AS ms
    ON mess.session_id = ms.web_pageview_id
    WHERE {ms_partition_statement}
    AND action = "filter_change"
    AND coalesce(dt, meta.dt) < "{limit_timestamp}"
),
mediasearch_quickview AS ( -- opened quickview during a search
    SELECT
        web_pageview_id AS session_id,
        MIN(coalesce(dt, meta.dt)) AS first_click_dt,
        1 AS clicked_result
    FROM mediasearch_sessions AS mess
    JOIN event.mediawiki_mediasearch_interaction AS ms
    ON mess.session_id = ms.web_pageview_id
    WHERE {ms_partition_statement}
    AND action = "result_click"
    AND coalesce(dt, meta.dt) > session_start_dt
    AND coalesce(dt, meta.dt) < "{limit_timestamp}"
    GROUP BY web_pageview_id
),
mediasearch_result_actions AS (
    -- We do all of these together because they're all based on mediasearch_quickview
    SELECT
        web_pageview_id AS session_id,
        -- clicked through to a file page
        MAX(IF(action = "quickview_more_details_click", 1, NULL)) AS file_page_click,
        -- clicked to copy the filename
        MAX(IF(action = "quickview_filename_copy", 1, NULL)) AS filename_copy,
        -- clicked to copy wikitext ()
        MAX(IF(action = "quickview_wikitext_link_copy", 1, NULL)) AS wikitext_copy,
        -- played media
        MAX(IF(action = "quickview_media_play", 1, NULL)) AS played_media
    FROM mediasearch_quickview AS mq
    JOIN event.mediawiki_mediasearch_interaction AS ms
    ON mq.session_id = ms.web_pageview_id
    WHERE {ms_partition_statement}
    AND coalesce(dt, meta.dt) > mq.first_click_dt
    AND coalesce(dt, meta.dt) < "{limit_timestamp}"
    GROUP BY web_pageview_id
),
median_click_position AS ( -- calculate across all valid sessions and result clicks
    SELECT
        TO_DATE(session_start_dt) AS log_date,
        percentile(
            search_result_position,
            0.5
        ) AS median_click_position
    FROM mediasearch_sessions AS mess
    JOIN event.mediawiki_mediasearch_interaction AS ms
    ON mess.session_id = ms.web_pageview_id
    WHERE {ms_partition_statement}
    AND action = "result_click"
    AND coalesce(dt, meta.dt) > session_start_dt
    AND coalesce(dt, meta.dt) < "{limit_timestamp}"
    GROUP BY TO_DATE(session_start_dt)
),
mediasearch_stats AS ( -- statistics for MediaSearch
    SELECT
        TO_DATE(session_start_dt) AS log_date,
        COUNT(1) AS num_mediasearch_sessions,
        COUNT(mf.used_filter) AS num_used_filter,
        COUNT(mq.clicked_result) AS num_result_clicked,
        COUNT(ma.file_page_click) AS num_filepage_clicked,
        COUNT(ma.filename_copy) AS num_filename_copied,
        COUNT(ma.wikitext_copy) AS num_wikitext_copied,
        COUNT(ma.played_media) AS num_media_played
    FROM mediasearch_sessions AS mess
    LEFT JOIN mediasearch_filters AS mf
    ON mess.session_id = mf.session_id
    LEFT JOIN mediasearch_quickview AS mq
    ON mess.session_id = mq.session_id
    LEFT JOIN mediasearch_result_actions AS ma
    ON mess.session_id = ma.session_id
    GROUP BY TO_DATE(session_start_dt)
)
INSERT INTO {aggregate_table}
SELECT
   mass.*,
   coalesce(mcp.median_click_position, 0.0) AS median_position_clicked
FROM mediasearch_stats AS mass
LEFT JOIN median_click_position AS mcp
ON mass.log_date = mcp.log_date
'''

In [8]:
try:
    spark.run(mediasearch_success_query.format(
        today = data_day,
        limit_timestamp = limit_timestamp.isoformat(),
        ess_partition_statement = make_partition_statement(previous_day, next_day, prefix = 'ess'),
        ms_partition_statement = make_partition_statement(previous_day, next_day, prefix = 'ms'),
        aggregate_table = search_success_table
    ))
except UnboundLocalError:
    # wmfdata currently (late Feb 2021) has an issue with DDL/DML SQL queries,
    # and so we ignore that error
    pass

PySpark executors will use /usr/bin/python3.7.
